# 네이버 뉴스에서 `비트코인` 관련 뉴스 수집하기 (`selenium`과 `newspaper3k` 사용)

시작하기 전.. Anaconda prompt에서 아래와 같이 입력하여 `selenium`을 설치하자.

```
conda install selenium
```

또한, 뉴스 내용을 수집하기 위한 `newspaper3k` 패키지를 설치하자.

```
pip install newspaper3k
```

# 1. 웹드라이버 정의 및 주소 접속

In [2]:
from selenium import webdriver

driver = webdriver.Chrome()

# 키워드, URL
keyword = '비트코인'
url = f'https://search.naver.com/search.naver?where=news&query={keyword}'

# url 접속
driver.get(url)
driver.implicitly_wait(2) # 2초 안에 웹페이지를 load하면 바로 넘어가거나, 2초를 기다림

# 2. 주소에서 스크롤을 n번 내림

In [3]:
from selenium.webdriver.common.keys import Keys
import time

# scroller
actions = driver.find_element('css selector', 'body')

actions.send_keys(Keys.END)
time.sleep(2)

actions.send_keys(Keys.END)
time.sleep(2)

actions.send_keys(Keys.END)
time.sleep(2)

# 3. press title에 해당하는 elements 모두 찾기

In [4]:
press_elements = driver.find_elements(
    'css selector', 
    'a.info.press')
title_elements = driver.find_elements(
    'css selector', 
    'a.news_tit')
len(press_elements), len(title_elements)

(40, 40)

# 4. 각 element에서 원하는 속성 및 텍스트 가져오기

In [7]:
# for title_el in title_elements:
    # ...
# for press_el in press_elements:
    # ...

title_list = []
url_list = []
press_list = []

for title_el, press_el in zip(title_elements,
                              press_elements):
    title = title_el.text
    url = title_el.get_attribute('href')
    press = press_el.text.replace('언론사 선정', '')
    title_list.append(title)
    url_list.append(url)
    press_list.append(press)
    # print(title)
    # print(url)
    # print(press)
    # print()

In [8]:
len(title_list), len(url_list), len(press_list)

(40, 40, 40)

# 5. 뉴스 주소마다 접속해서 뉴스의 datetime, text 가져오기

In [9]:
from newspaper import Article

In [13]:
text_list = []
datetime_list = []

for url in url_list:
    article = Article(url, language='ko')
    article.download()
    article.parse()
    text_list.append(article.text)
    datetime_list.append(article.publish_date)
    time.sleep(0.5)
    # print(article.title)
    # print(article.authors)
    # print(article.publish_date)
    # print(article.text)
len(text_list), len(datetime_list)

(40, 40)

# 6. 테이블로 변환

In [14]:

import pandas as pd

# 뉴스 제목 데이터프레임 생성
df = pd.DataFrame({'url': url_list,
                   'press': press_list,
                   'datetime': datetime_list,
                   'title': title_list,
                   'text': text_list
                  })
df

,url,press,datetime,title,text
0,https://www.donga.com/news/Politics/article/al...,동아일보,2025-03-27 13:00:00,국회의원 재산 평균 26억… 부동산 13채-비트코인-페라리 신고도,[고위공직자 재산공개]\n\n안철수 1367억-박덕흠 535억 신고\n\n박정 36...
1,https://www.newsis.com/view/NISX20250327_00031...,뉴시스,2025-03-27 09:20:24+09:00,"비트코인, 자동차 관세 발표에 1억2800만원대 횡보",[워싱턴=AP/뉴시스]도널드 트럼프 미국 대통령이 26일(현지 시간) 백악관 이스트...
2,https://economist.co.kr/article/view/ecn202503...,이코노미스트,None,"비트코인, 게임스톱 비축 선언에도 무반응…1억2800만원대 머물러",비트코인 [사진 연합뉴스]\n\n[이코노미스트 우승민 기자] 비트코인이 1억2800...
3,https://zdnet.co.kr/view/?no=20250327111847,지디넷코리아,2025-03-27 13:21:48+09:00,비트코인 시세 반등세…기대와 신중론 엇갈리는 가상자산 투자자,지난 3개월간 하락세를 이어온 비트코인 가격이 최근 반등 흐름을 보이며 시장 기대감...
4,https://biz.heraldcorp.com/article/10450093?re...,헤럴드경제,2025-03-26 07:32:33+09:00,이익 반토막에도 “비트코인 살게” 한마디에 시들하던 ‘밈주식’ 주가...,[헤럴드경제=김우영 기자] 미국 비디오게임 업체 게임스탑(GameStop)이 보유 ...
5,http://weekly.chosun.com/news/articleView.html...,주간조선,2025-03-27 09:30:00+09:00,비트코인·럭셔리카·다이아몬드… 국회의원 정기 재산신고,"photo Freepik\n\n국회의원들이 보유한 가상자산과 고가 예술품, 슈퍼카 ..."
6,http://www.edaily.co.kr/news/newspath.asp?news...,이데일리,2025-03-25 08:27:12+09:00,트럼프 관세 완화에 비트코인 상승…8만 7000달러대 거래,[이데일리 강민구 기자] 도널드 트럼프 미국 대통령이 관세정책 완화를 시사하면서 뉴...
7,https://www.sedaily.com/NewsView/2GQEO76JKW,서울경제,2025-03-27 03:00:00+09:00,"""비트코인 올해 20만 달러 간다""…'부자아빠', 역사상 가장 큰 기회라는...","viewer 로버트 기요사키. 기요사키 SNS\n\n< 저작권자 ⓒ 서울경제, 무단..."
8,https://www.hankyung.com/article/2025032579331,한국경제,2025-03-25 17:59:10+09:00,"""美 정부·기관 앞다퉈 비트코인 보유…韓도 주도권 경쟁 나서야""","""올해 글로벌 기관투자가 비트코인 자산 편입 늘 것""\n\n\n\n가상자산 투자 인..."
9,https://www.news1.kr/finance/blockchain-fintec...,뉴스1,None,美증시 요동에도 비트코인 견고…게임스탑도 매수 선언[코인브리핑],
